# Fair Binary Classification with SearchFair on CelebA and Adult

Here, we show how to use SearchFair on two datasets: CelebA and Adult

## Imports

We start by importing SearchFair from the installed package.

In [1]:
from searchfair import SearchFair

Second, we load some necessary methods and numpy.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np

# We are ignoring cvxpys warning about disciplined programming rules. 
import warnings
warnings.filterwarnings('ignore')

# The CelebA dataset

On the Celebrity Faces dataset we are given descriptions of celebrity faces, with 40 binary attributes. Here, we use the Attribute 'Smiling' as the class label, and sex as the sensitive attribute. 

In [3]:
import get_real_data as get_data

# Load Data
x_data, y_data, s_data = get_data.get_celebA_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 800
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)

Here are some basic information about the dataset:

In [4]:
import utils as ut
ut.print_data_stats(s_data, y_data)

Total data points: 202599
# non-protected examples: 118165
# protected examples: 84434
# non-protected examples in positive class: 63871 (54.1%)
# protected examples in positive class: 33798 (40.0%)


## Creating a SearchFair Model

### Demographic Parity

To learn a classifier with SearchFair, we need to choose a kernel between 'linear' and 'rbf', and we need to choose a fairness notion - either Demographic Parity (DDP) or Equality of Opportunity (DEO). 

In [5]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True # True = SearchFair output, 2 = show also solver progress

Let us choose a regularization parameter and then fit a model with the default settings. 

In [6]:
# Regularization Parameter beta
reg_beta = 0.0001
linear_model_DDP = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
linear_model_DDP.fit(x_train, y_train, s_train=s_train)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DDP = 0.2239 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = -0.7728 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = -0.4568 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DDP = 0.0027 with lambda = 0.2500
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.2500 with fairness 0.0027----------


SearchFair(reg_beta=0.0001, verbose=True)

To print out the Accuracy and the fairness notions Demographic Parity and Equality of Opportuniy, we define the following function. 

In [7]:
def print_clf_stats(model, x_train, x_test, y_train, y_test, s_train, s_test):
    train_acc = ut.get_accuracy(np.sign(model.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(model.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(model.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(model.predict(x_train), y_train, s_train)

    print(10*'-'+"Train"+10*'-')
    print("Accuracy: %0.4f%%" % (train_acc * 100))
    print("DDP: %0.4f%%" % (train_DDP * 100), "DEO: %0.4f%%" % (train_DEO * 100))
    print(10*'-'+"Test"+10*'-')
    print("Accuracy: %0.4f%%" % (test_acc * 100))
    print("DDP: %0.4f%%" % (test_DDP * 100), "DEO: %0.4f%%" % (test_DEO * 100))

Now lets see, if we obtained a fair classifier with respect to the fairness notions we specified. 

In [8]:
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 82.8750%
DDP: 0.2735% DEO: 6.2114%
----------Test----------
Accuracy: 77.8398%
DDP: -1.9669% DEO: 10.8829%


Probably, you will see that the train DDP is small, and SearchFair succeed to find a fair classifier. The test DDP might or might not be close to 0. This is due to the small number of points which are used to reduce running time for this example notebook. Go ahead and try it with more points!

### Equality of Opportunity

Now we try the same using a more complex rbf kernel, and we try to improve Equality of Opportunity this time. 

In [9]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = True

# Regularization Parameter beta
reg_beta = 0.0001
rbf_model_DEO = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
rbf_model_DEO.fit(x_train, y_train, s_train=s_train)

# Evaluate model
print_clf_stats(rbf_model_DEO, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.1154 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.9184 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DEO = -0.8734 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DEO = -0.0880 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DEO = 0.0347 with lambda = 0.1250
----------Iteration #3----------
Testing new Lambda: 0.1875
Obtained: DEO = -0.0003 with lambda = 0.1875
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.1875 with fairness -0.0003----------
----------Train----------
Accuracy: 89.2500%
DDP: 7.5450% DEO: -0.0314%
----------Test----------
Accuracy: 84.1833%
DDP: 13.9705% DEO: 11.9893%


### Cross Validation - GridSearch

It is also possible to use GridSearchCV for the regularization paramter beta, and, if used, the width of the rbf kernel. But running this might take some time.

In [10]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = False

cv_model = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_magn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model,cv_params, cv=3, verbose=2, n_jobs=1, scoring='accuracy', refit=True)
grid_clf.fit(x_train, y_train, s_train=s_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] gamma=0.01, reg_beta=0.0001 .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...................... gamma=0.01, reg_beta=0.0001, total= 1.1min
[CV] gamma=0.01, reg_beta=0.0001 .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV] ...................... gamma=0.01, reg_beta=0.0001, total=  59.1s
[CV] gamma=0.01, reg_beta=0.0001 .....................................
[CV] ...................... gamma=0.01, reg_beta=0.0001, total=  58.0s
[CV] gamma=0.01, reg_beta=0.001 ......................................
[CV] ....................... gamma=0.01, reg_beta=0.001, total=  52.5s
[CV] gamma=0.01, reg_beta=0.001 ......................................
[CV] ....................... gamma=0.01, reg_beta=0.001, total=  56.1s
[CV] gamma=0.01, reg_beta=0.001 ......................................
[CV] ....................... gamma=0.01, reg_beta=0.001, total=  59.0s
[CV] gamma=0.01, reg_beta=0.01 .......................................
[CV] ........................ gamma=0.01, reg_beta=0.01, total=  47.7s
[CV] gamma=0.01, reg_beta=0.01 .......................................
[CV] ........................ gamma=0.01, reg_beta=0.01, total=  50.1s
[CV] gamma=0.01, reg_beta=0.01 .......................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 23.6min finished


GridSearchCV(cv=3, estimator=SearchFair(kernel='rbf'), n_jobs=1,
             param_grid={'gamma': [0.01, 0.02631578947368421, 0.1],
                         'reg_beta': [0.0001, 0.001, 0.01]},
             scoring='accuracy', verbose=2)

In [11]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 87.7500%
DDP: 0.4396% DEO: -4.0625%
----------Test----------
Accuracy: 82.1773%
DDP: 5.5963% DEO: 7.5117%


## Adult dataset

In the fairness literature, the adult dataset is a very popular dataset. It contains US census data from 1994, where the class label indicates if the income is higher or lower than 50.000$. The binary sensitive attribute here, is the sex.

In [ ]:
# Load Data
x_data, y_data, s_data = get_data.get_adult_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 1000
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
ut.print_data_stats(s_data, y_data)

If you want, you can also try SearchFair on Adult. 

In [ ]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = True

cv_model = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_m^agn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model,cv_params, cv=3, verbose=2, n_jobs=1, scoring='accuracy')
grid_clf.fit(x_train, y_train, s_train=s_train)

In [ ]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)